## Data Challenge | Data cleaning and wrangling blueprint

Ths first notebook processes the data cleaning and wrangling I wanted to do before starting to analyse and work with the provided dataset.  
Null values are treated, in particular the datas are filtered by those IDs whose age is +18 years old, since the survey is conducted only on legally adults.    
Rows contantaining the questions not relevant for my analysis are dropped.    
Finally columns are casted into the correct datatype.

In [1]:
# import libraries
import pandas as pd
import numpy as np

## Loading and cleaning Indagine data

In [2]:
# Load Indagine data labels
indagine_label = pd.read_excel('data/dataset_indagine_stati_animo.xlsx', sheet_name='Labels')
print(indagine_label.shape)
indagine_label.head(3)

(8128, 108)


,ID_Rispondente,B11_Q62_A_1,B11_Q62_A_2,B11_Q62_A_3,B11_Q62_A_4,B11_Q62_A_5,B11_Q62_A_6,B11_Q62_A_7,B11_Q62_A_8,B11_Q62_A_9,...,p_Q14_A_2,p_Q14_A_3,p_Q14_A_4,p_Q14_A_5,p_Q14_A_6,p_Q14_A_7,p_Q14_A_8,p_Q14_A_9,Mese,peso
0,3936511661,Molto,Un po' sì e un po' no,Poco,Abbastanza,Poco,Molto,Un po' sì e un po' no,Molto,Abbastanza,...,5 volte,Due volte,3/ 4 volte,3/ 4 volte,Mai o meno di una volta,Due volte,Due volte,Mai o meno di una volta,2,0.581195
1,164860483,Un po' sì e un po' no,Per niente,Poco,Abbastanza,Poco,Abbastanza,Per niente,Abbastanza,Abbastanza,...,3/ 4 volte,Mai o meno di una volta,Due volte,Mai o meno di una volta,Una volta alla settimana,3/ 4 volte,3/ 4 volte,Mai o meno di una volta,2,5.458265
2,221250719,Molto,Per niente,Per niente,Abbastanza,Per niente,Abbastanza,Per niente,Abbastanza,Abbastanza,...,Mai o meno di una volta,Mai o meno di una volta,Più di 5 volte alla settimana,Mai o meno di una volta,Più di 5 volte alla settimana,3/ 4 volte,3/ 4 volte,Mai o meno di una volta,2,5.458265


In [3]:
# Load Indagine data values
indagine_value = pd.read_excel('data/dataset_indagine_stati_animo.xlsx', sheet_name='Values')
print(indagine_label.shape)
indagine_value.head(3)

(8128, 108)


,ID_Rispondente,B11_Q62_A_1,B11_Q62_A_2,B11_Q62_A_3,B11_Q62_A_4,B11_Q62_A_5,B11_Q62_A_6,B11_Q62_A_7,B11_Q62_A_8,B11_Q62_A_9,...,p_Q14_A_2,p_Q14_A_3,p_Q14_A_4,p_Q14_A_5,p_Q14_A_6,p_Q14_A_7,p_Q14_A_8,p_Q14_A_9,Mese,peso
0,3936511661,5,3,2,4,2,5,3,5,4,...,5.0,3.0,4.0,4.0,1.0,3.0,3.0,1.0,2,0.581195
1,164860483,3,1,2,4,2,4,1,4,4,...,4.0,1.0,3.0,1.0,2.0,4.0,4.0,1.0,2,5.458265
2,221250719,5,1,1,4,1,4,1,4,4,...,1.0,1.0,6.0,1.0,6.0,4.0,4.0,1.0,2,5.458265


In [4]:
# Load Indagine data structure --> indagine_lookup (data mapping)
indagine_lookup = pd.read_excel('data/dataset_indagine_stati_animo.xlsx', sheet_name='Struttura dei dati', skiprows=5, nrows=indagine_value.shape[1])
print(indagine_lookup.shape)

(108, 3)


In [5]:
# Cleaning of the Etichetta variable in the Indagine data structure DataFrame --> to generate the Domanda variable
indagine_lookup['Domanda'] = indagine_lookup['Etichetta'].apply(lambda st: st[st.find("Question: ")+10:np.min([st.find("]"), st.find("<span ")])] if st.find("Question: ") >= 0 else st)
indagine_lookup['Domanda'] = indagine_lookup['Domanda'].apply(lambda st: st[:st.find("<span ")] if st.find("<span ") >= 0 else st)
indagine_lookup['Domanda'] = indagine_lookup['Domanda'].apply(lambda st: st[:st.find("<br />")] if st.find("<br />") >= 0 else st)

In [6]:
# Cleaning of the Etichetta variable in the Indagine data structure DataFrame --> to generate the Topic variable
indagine_lookup['Topic'] = indagine_lookup['Etichetta'].apply(lambda st: st[st.find("Topic: ")+7:st.find("Question: ")-1] if st.find("Topic: ") >= 0 else st[:st.find("[Question: ")-1] if st.find("[Question: ") >= 0 else "")

In [7]:
# replace values in column 'Domanda' of the lookup table with the ones in the list
for el in ['Age', 'Comune', 'Provincia', 'Regione', 'peso']: 
    indagine_lookup.loc[indagine_lookup['Variabile']==el,'Domanda'] = el

In [8]:
# Check top 3 rows of indagine_lookup DF
indagine_lookup.head(3)

,Variabile,Posizione,Etichetta,Domanda,Topic
0,ID_Rispondente,1,Codice identificativo della persona intervistata,Codice identificativo della persona intervistata,
1,B11_Q62_A_1,2,[Topic: Attivo/ intraprendente Question: Di se...,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
2,B11_Q62_A_2,3,[Topic: Turbato/ Agitato Question: Di seguito ...,Di seguito trovi una serie di parole che indic...,Turbato/ Agitato


In [9]:
# Melting the indagine_value DF --> transposing the data from horizontal to vertical
indagine_value_melt = pd.melt(indagine_value, id_vars='ID_Rispondente', var_name='domanda', value_name='values_risposta')
indagine_value_melt.shape

(869696, 3)

In [10]:
# Melting the indagine_label DF --> transposing the data from horizontal to vertical
indagine_label_melt = pd.melt(indagine_label, id_vars='ID_Rispondente', var_name='domanda', value_name='labels_risposta')
indagine_label_melt.shape

(869696, 3)

In [11]:
# Inner joining the indagine_label_melt and indagine_value_melt --> indagine_melt
indagine_melt = pd.merge(indagine_label_melt, indagine_value_melt, how='inner', on=['ID_Rispondente','domanda'])
print(indagine_melt.shape)
indagine_melt.head()

(869696, 4)


,ID_Rispondente,domanda,labels_risposta,values_risposta
0,3936511661,B11_Q62_A_1,Molto,5
1,164860483,B11_Q62_A_1,Un po' sì e un po' no,3
2,221250719,B11_Q62_A_1,Molto,5
3,89508748,B11_Q62_A_1,Un po' sì e un po' no,3
4,98143773,B11_Q62_A_1,Poco,2


In [12]:
# Left joining the indagine_melt with the indagine_lookup (data mapping) DF --> generating the final indagine DF
indagine = pd.merge(indagine_melt, indagine_lookup[['Variabile', 'Domanda', 'Topic']], how='left', left_on='domanda', right_on='Variabile')
print(indagine.shape)
indagine.head()

(869696, 7)


,ID_Rispondente,domanda,labels_risposta,values_risposta,Variabile,Domanda,Topic
0,3936511661,B11_Q62_A_1,Molto,5,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
1,164860483,B11_Q62_A_1,Un po' sì e un po' no,3,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
2,221250719,B11_Q62_A_1,Molto,5,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
3,89508748,B11_Q62_A_1,Un po' sì e un po' no,3,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
4,98143773,B11_Q62_A_1,Poco,2,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente


In [13]:
# check unique values in 'Domanda' column
indagine['Domanda'].unique()

array(['Di seguito trovi una serie di parole che indicano diverse sensazioni ed emozioni. Per ogni parola indica quanto ti senti così oggi',
       'Quando pensi al futuro, fra un anno ti vedi più o meno...',
       'Come vedi la tua vita nei prossimi 12 mesi?Quanto ritieni probabile entro i prossimi 12 mesi di...',
       "Quali sono i tre stati d'animo principali che caratterizzano le relazioni che vivi in casa tua?",
       "Quali sono i tre stati d'animo principali che caratterizzano le relazioni che vivi con le persone che abitano vicino a te (per es., nel tuo quartiere/ paese)?",
       "Quando pensi all'attualità italiana (politica, economia, cronaca, cultura, etc.) quali sono i tre stati d'animo principali che accompagnano la tua lettura degli eventi?",
       "E, infine, quando pensi alle vicende del mondo (politica, economia, progetti, eventi, etc.) quali sono i tre stati d'animo principali che accompagnano la tua lettura degli eventi?",
       'In generale, quanto sei soddis

In [14]:
# filter rows 'indagine' DF for those datas that I want to see as specific columns and assign them to a 'info' DF
info = indagine[indagine['Domanda'].isin(['Tempo di compilazione delle sole risposte complete',
       'Secondi per risposta',
       'Data e ora di compilazione del questionario', 'Sesso', 'Age',
       'Età del rispondente in sei fasce',
       "Qual è stato l'ultimo titolo di studio che hai conseguito?",
       'CAP', 'Comune', 'Provincia', 'Regione',
       'Classe dimensione centro', 'Numero di abitanti',
       'Area Nielsen (4 aree)', 'Area Nielsen (5 aree)', 'Mese in cui è stato compilato il questionario', 'peso']) ]
info.head()

,ID_Rispondente,domanda,labels_risposta,values_risposta,Variabile,Domanda,Topic
658368,3936511661,B11_tempo,278.183,278.183,B11_tempo,Tempo di compilazione delle sole risposte comp...,
658369,164860483,B11_tempo,189.371,189.371,B11_tempo,Tempo di compilazione delle sole risposte comp...,
658370,221250719,B11_tempo,130.191,130.191,B11_tempo,Tempo di compilazione delle sole risposte comp...,
658371,89508748,B11_tempo,155.58,155.58,B11_tempo,Tempo di compilazione delle sole risposte comp...,
658372,98143773,B11_tempo,117.973,117.973,B11_tempo,Tempo di compilazione delle sole risposte comp...,


In [15]:
# pivot info df to transform rows into columns
info = info[['ID_Rispondente','Domanda','labels_risposta']]
info_pivot = info.pivot(index='ID_Rispondente', columns='Domanda', values='labels_risposta')
info_pivot.reset_index(drop=False, inplace=True)
print(info_pivot.shape)
info_pivot.head(5)

(8128, 18)


Domanda,ID_Rispondente,Age,Area Nielsen (4 aree),Area Nielsen (5 aree),CAP,Classe dimensione centro,Comune,Data e ora di compilazione del questionario,Età del rispondente in sei fasce,Mese in cui è stato compilato il questionario,Numero di abitanti,Provincia,Qual è stato l'ultimo titolo di studio che hai conseguito?,Regione,Secondi per risposta,Sesso,Tempo di compilazione delle sole risposte complete,peso
0,256848,39.0,Area 3,Isole,8100.0,Tra 30 e 100mila,NUORO,44976.441655,35-44,2,36678.0,NU,Laurea o dottorato,SAR,8.16775,Una donna,98.013,0.218331
1,337988,56.0,Area 1,Nordovest,24057.0,Tra 10 e 30mila,MARTINENGO,45058.745671,55-64,5,10524.0,BG,Diploma di scuola media inferiore,LOM,12.13825,Un uomo,145.659,1.211879
2,371674,46.0,Area 2,Nordest,34139.0,Più di 100mila,TRIESTE,44985.510521,45-54,2,211184.0,TS,Laurea o dottorato,FVG,11.990917,Una donna,143.891,0.218331
3,379221,77.0,Area 4,Sud,85044.0,Tra 10 e 30mila,LAURIA,45038.741539,65+,4,13801.0,PZ,Diploma di scuola media superiore,BAS,19.758083,Un uomo,237.097,2.944145
4,514191,73.0,Area 4,Sud,70123.0,Più di 100mila,BARI,44995.776377,65+,3,316532.0,BA,Diploma di scuola media inferiore,PUG,21.511917,Un uomo,258.143,2.047296


In [16]:
info_pivot.columns

Index(['ID_Rispondente', 'Age', 'Area Nielsen (4 aree)',
       'Area Nielsen (5 aree)', 'CAP', 'Classe dimensione centro', 'Comune',
       'Data e ora di compilazione del questionario',
       'Età del rispondente in sei fasce',
       'Mese in cui è stato compilato il questionario', 'Numero di abitanti',
       'Provincia',
       'Qual è stato l'ultimo titolo di studio che hai conseguito?', 'Regione',
       'Secondi per risposta', 'Sesso',
       'Tempo di compilazione delle sole risposte complete', 'peso'],
      dtype='object', name='Domanda')

In [17]:
# drop 'indagine' DF rows that will be columns
indagine.drop(indagine[indagine['Domanda'].isin(['Tempo di compilazione delle sole risposte complete',
       'Secondi per risposta',
       'Data e ora di compilazione del questionario', 'Sesso', 'Age',
       'Età del rispondente in sei fasce',
       "Qual è stato l'ultimo titolo di studio che hai conseguito?",
       'CAP', 'Comune', 'Provincia', 'Regione',
       'Classe dimensione centro', 'Numero di abitanti',
       'Area Nielsen (4 aree)', 'Area Nielsen (5 aree)', 'Mese in cui è stato compilato il questionario', 'peso']) ].index,inplace=True)

In [18]:
# merge 'indagine' DF with 'info_pivot' DF containing new columns (ex 'indagine' rows)
indagine = pd.merge(indagine, info_pivot, how='left', on='ID_Rispondente')
print(indagine.shape)
indagine.head(3)

(731520, 24)


,ID_Rispondente,domanda,labels_risposta,values_risposta,Variabile,Domanda,Topic,Age,Area Nielsen (4 aree),Area Nielsen (5 aree),...,Età del rispondente in sei fasce,Mese in cui è stato compilato il questionario,Numero di abitanti,Provincia,Qual è stato l'ultimo titolo di studio che hai conseguito?,Regione,Secondi per risposta,Sesso,Tempo di compilazione delle sole risposte complete,peso
0,3936511661,B11_Q62_A_1,Molto,5,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,14.0,Area 4,Sud,...,NaN,2,14894.0,BT,Laurea o dottorato,PUG,23.181917,Una donna,278.183,0.581195
1,164860483,B11_Q62_A_1,Un po' sì e un po' no,3,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,17.0,Area 4,Sud,...,18-24,2,7615.0,PE,Diploma di scuola media inferiore,ABR,15.780917,Un uomo,189.371,5.458265
2,221250719,B11_Q62_A_1,Molto,5,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,20.0,Area 4,Sud,...,18-24,2,56929.0,BA,Diploma di scuola media superiore,PUG,10.84925,Un uomo,130.191,5.458265


In [19]:
# drop redondant columns and keep only te relevant ones
indagine.drop(columns='domanda', inplace=True)
indagine = indagine[['ID_Rispondente', 'Variabile', 'Domanda', 'Topic', 'labels_risposta', 'values_risposta', 'Age', 'CAP', 'Provincia', 
                     'Numero di abitanti', 'Qual è stato l\'ultimo titolo di studio che hai conseguito?', 
                     'Data e ora di compilazione del questionario', 'Secondi per risposta', 'Sesso', 'peso']]

In [20]:
# rename column
indagine.rename(columns={'Qual è stato l\'ultimo titolo di studio che hai conseguito?':'Ultimo titolo di studio conseguito'}, inplace=True)
indagine.head()

,ID_Rispondente,Variabile,Domanda,Topic,labels_risposta,values_risposta,Age,CAP,Provincia,Numero di abitanti,Ultimo titolo di studio conseguito,Data e ora di compilazione del questionario,Secondi per risposta,Sesso,peso
0,3936511661,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,Molto,5,14.0,76017.0,BT,14894.0,Laurea o dottorato,44977.442708,23.181917,Una donna,0.581195
1,164860483,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,Un po' sì e un po' no,3,17.0,65014.0,PE,7615.0,Diploma di scuola media inferiore,44978.840463,15.780917,Un uomo,5.458265
2,221250719,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,Molto,5,20.0,70032.0,BA,56929.0,Diploma di scuola media superiore,44971.796343,10.84925,Un uomo,5.458265
3,89508748,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,Un po' sì e un po' no,3,21.0,87060.0,CS,1281.0,Diploma di scuola media superiore,44984.775023,12.965,Una donna,5.458265
4,98143773,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,Poco,2,21.0,63822.0,FM,16062.0,Diploma di scuola media superiore,44976.777766,9.831083,Una donna,5.458265


In [21]:
# check null values for all columns
indagine.isnull().sum()

ID_Rispondente                                     0
Variabile                                          0
Domanda                                            0
Topic                                              0
labels_risposta                                 7942
values_risposta                                 7942
Age                                              270
CAP                                              270
Provincia                                      42210
Numero di abitanti                              5490
Ultimo titolo di studio conseguito               270
Data e ora di compilazione del questionario        0
Secondi per risposta                            7830
Sesso                                            270
peso                                               0
dtype: int64

In [22]:
# check unique values for 'Domanda' to drop which questions are not relevant for my analysis
indagine['Domanda'].unique()

array(['Di seguito trovi una serie di parole che indicano diverse sensazioni ed emozioni. Per ogni parola indica quanto ti senti così oggi',
       'Quando pensi al futuro, fra un anno ti vedi più o meno...',
       'Come vedi la tua vita nei prossimi 12 mesi?Quanto ritieni probabile entro i prossimi 12 mesi di...',
       "Quali sono i tre stati d'animo principali che caratterizzano le relazioni che vivi in casa tua?",
       "Quali sono i tre stati d'animo principali che caratterizzano le relazioni che vivi con le persone che abitano vicino a te (per es., nel tuo quartiere/ paese)?",
       "Quando pensi all'attualità italiana (politica, economia, cronaca, cultura, etc.) quali sono i tre stati d'animo principali che accompagnano la tua lettura degli eventi?",
       "E, infine, quando pensi alle vicende del mondo (politica, economia, progetti, eventi, etc.) quali sono i tre stati d'animo principali che accompagnano la tua lettura degli eventi?",
       'In generale, quanto sei soddis

In [23]:
# drop irrelevant questions in 'Domanda' column
indagine.drop(indagine[(indagine['Domanda'] == 'Infine, se vuoi ancora dirci qualcosa sul questionario e l\'intervista, scrivi quello che vuoi in questo spazio:') |
                       (indagine['Domanda'] == 'Quante volte in una settimana ti capita di svolgere le seguenti attività?') | 
                       (indagine['Domanda'] == 'Come valuti complessivamente questa intervista?') ].index, inplace=True)
print(indagine.shape)

(642112, 15)


In [24]:
print(indagine.isnull().sum())
# drop null values for column 'Age', since the survey should consider only people +18 years old
indagine.dropna(subset='Age', axis=0, inplace=True)

ID_Rispondente                                     0
Variabile                                          0
Domanda                                            0
Topic                                              0
labels_risposta                                    0
values_risposta                                    0
Age                                              237
CAP                                              237
Provincia                                      37051
Numero di abitanti                              4819
Ultimo titolo di studio conseguito               237
Data e ora di compilazione del questionario        0
Secondi per risposta                            6873
Sesso                                            237
peso                                               0
dtype: int64


In [25]:
print(indagine.shape)
indagine.isnull().sum()

(641875, 15)


ID_Rispondente                                     0
Variabile                                          0
Domanda                                            0
Topic                                              0
labels_risposta                                    0
values_risposta                                    0
Age                                                0
CAP                                                0
Provincia                                      36814
Numero di abitanti                              4582
Ultimo titolo di studio conseguito                 0
Data e ora di compilazione del questionario        0
Secondi per risposta                            6873
Sesso                                              0
peso                                               0
dtype: int64

In [26]:
# check columns datatype
indagine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 641875 entries, 0 to 642111
Data columns (total 15 columns):
 #   Column                                       Non-Null Count   Dtype 
---  ------                                       --------------   ----- 
 0   ID_Rispondente                               641875 non-null  int64 
 1   Variabile                                    641875 non-null  object
 2   Domanda                                      641875 non-null  object
 3   Topic                                        641875 non-null  object
 4   labels_risposta                              641875 non-null  object
 5   values_risposta                              641875 non-null  object
 6   Age                                          641875 non-null  object
 7   CAP                                          641875 non-null  object
 8   Provincia                                    605061 non-null  object
 9   Numero di abitanti                           637293 non-null  object
 

## Cast columns into the correct datatypes

In [27]:
# cast col 'values_risposta' as Integer
indagine['values_risposta'] = [str(i) for i in indagine['values_risposta']]
indagine['values_risposta'] = [i.replace('.0','') for i in indagine['values_risposta']]
indagine['values_risposta'] = indagine['values_risposta'].astype(dtype='int64' , errors='ignore')

In [28]:
# cast col 'Age' as Integer
indagine['Age'] = [str(i) for i in indagine['Age']]
indagine['Age'] = [i.replace('.0','') for i in indagine['Age']]
indagine['Age'] = indagine['Age'].astype(dtype='int64' , errors='ignore')

In [29]:
# cast col 'CAP' as String
indagine['CAP'] = [str(i) for i in indagine['CAP']]
indagine['CAP'] = [i.replace('.0','') for i in indagine['CAP']]

In [30]:
# cast col 'Numero di abitanti' as Integer
indagine['Numero di abitanti'] = [str(i) for i in indagine['Numero di abitanti']]
indagine['Numero di abitanti'] = [i.replace('.0','') for i in indagine['Numero di abitanti']]
indagine['Numero di abitanti'] = indagine['Numero di abitanti'].astype(dtype='int64' , errors='ignore')

In [31]:
# cast col 'Data e ora di compilazione dle qustionario' as Datetime value. Import specific library to transform xls datetime into readable date
import xlrd
indagine['Data e ora di compilazione del questionario'] = [xlrd.xldate_as_datetime(i, 0) for i in indagine['Data e ora di compilazione del questionario']]
indagine['Data e ora di compilazione del questionario']

0        2023-02-20 10:37:30
1        2023-02-21 20:10:16
2        2023-02-14 19:06:44
3        2023-02-27 18:36:02
4        2023-02-19 18:39:59
                 ...        
642107   2023-09-25 12:20:32
642108   2023-09-02 07:27:00
642109   2023-09-12 18:41:24
642110   2023-09-05 17:42:39
642111   2023-09-10 17:28:29
Name: Data e ora di compilazione del questionario, Length: 641875, dtype: datetime64[ns]

In [32]:
indagine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 641875 entries, 0 to 642111
Data columns (total 15 columns):
 #   Column                                       Non-Null Count   Dtype         
---  ------                                       --------------   -----         
 0   ID_Rispondente                               641875 non-null  int64         
 1   Variabile                                    641875 non-null  object        
 2   Domanda                                      641875 non-null  object        
 3   Topic                                        641875 non-null  object        
 4   labels_risposta                              641875 non-null  object        
 5   values_risposta                              641875 non-null  int64         
 6   Age                                          641875 non-null  int64         
 7   CAP                                          641875 non-null  object        
 8   Provincia                                    605061 non-null  ob

In [33]:
# filter 'indagine' dataframe for people whose 'Age' is > = 18 years old. Re-assign 'indagine' filtered to a new dataframe (same name 'indagine')
indagine = indagine[indagine['Age'] >= 18]
print(indagine.shape)
indagine.isnull().sum()

(641480, 15)


ID_Rispondente                                     0
Variabile                                          0
Domanda                                            0
Topic                                              0
labels_risposta                                    0
values_risposta                                    0
Age                                                0
CAP                                                0
Provincia                                      36814
Numero di abitanti                                 0
Ultimo titolo di studio conseguito                 0
Data e ora di compilazione del questionario        0
Secondi per risposta                            6873
Sesso                                              0
peso                                               0
dtype: int64

## Loading and cleaning Profilo raw data provided to add more infos at Indatine data

In [34]:
# Load Profilo data labels
profilo_label = pd.read_excel('data/dataset_profilo_intervistati.xlsx', sheet_name='Labels')
print(profilo_label.shape)
profilo_label.head(3)

(12853, 293)


,ID_Rispondente,B01_Q64_1,B01_Q64_2,B01_Q64_3,B01_Q64_4,B01_Q64_5,B01_Q64_6,B01_Q64_7,B01_Q64_8,B01_Q64_9,...,B10_Q66_7,B10_Q66_8,B10_Q66_9,B10_Q66_10,B10_Q66_11,B10_Q66_12,B10_Q66_13,B10_Q66_14,B10_Q66_15,B10_Q66_16
0,256848,Selected,Selected,Selected,Selected,Not Selected,Selected,Not Selected,Selected,Not Selected,...,NaN,Not Selected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Selected
1,337988,Selected,Selected,Selected,Selected,Not Selected,Selected,Not Selected,Selected,Not Selected,...,Not Selected,Not Selected,Not Selected,Not Selected,Not Selected,NaN,Not Selected,Not Selected,Not Selected,Not Selected
2,369486,Selected,Selected,Selected,Selected,Not Selected,Selected,Not Selected,Selected,Not Selected,...,NaN,Selected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Selected


In [35]:
# Load Profilo data values
profilo_value = pd.read_excel('data/dataset_profilo_intervistati.xlsx', sheet_name='Values')
print(profilo_label.shape)
profilo_value.head(3)

(12853, 293)


,ID_Rispondente,B01_Q64_1,B01_Q64_2,B01_Q64_3,B01_Q64_4,B01_Q64_5,B01_Q64_6,B01_Q64_7,B01_Q64_8,B01_Q64_9,...,B10_Q66_7,B10_Q66_8,B10_Q66_9,B10_Q66_10,B10_Q66_11,B10_Q66_12,B10_Q66_13,B10_Q66_14,B10_Q66_15,B10_Q66_16
0,256848,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,337988,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
2,369486,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [36]:
# Load Profilo data structure --> profilo_lookup (data mapping)
profilo_lookup = pd.read_excel('data/dataset_profilo_intervistati.xlsx', sheet_name='Struttura dei dati', skiprows=5, nrows=profilo_value.shape[1])
print(profilo_lookup.shape)

(293, 3)


In [37]:
# Cleaning of the Etichetta variable in the Profilo data structure DataFrame --> to generate the Domanda variable
profilo_lookup['Domanda'] = profilo_lookup['Etichetta'].apply(lambda st: st[st.find("Question: ")+10:np.min([st.find("]"), st.find("<br />")])] if st.find("Question: ") >= 0 else st)

In [38]:
# Cleaning of the Etichetta variable in the Profilo data structure DataFrame --> to generate the Topic variable
profilo_lookup['Topic'] = profilo_lookup['Etichetta'].apply(lambda st: st[st.find("/> ")+3:st.find(" [Question:")] if st.find("<img src=") >= 0 else st[0:st.find(" [Question:")] if st.find(" [Question:]") else "")
profilo_lookup['Topic'] = profilo_lookup['Topic'].apply(lambda st: st[st.find("[Topic: ")+8:st.find(" Question:")] if st.find("[Topic: ") >= 0 else st)
profilo_lookup['Topic'] = profilo_lookup['Topic'].apply(lambda st: st[0:st.find("<br />")] if st.find("<br />") >= 0 else st)

In [39]:
# Check top 3 rows of profilo_lookup DF
profilo_lookup.head(3)

,Variabile,Posizione,Etichetta,Domanda,Topic
0,ID_Rispondente,1,Codice identificativo della persona intervistata,Codice identificativo della persona intervistata,Codice identificativo della persona intervistat
1,B01_Q64_1,2,"<img src=""https://surveys.opline.it/media/logo...",Quali di questi strumenti di social networking...,Twitter
2,B01_Q64_2,3,"<img src=""https://surveys.opline.it/media/logo...",Quali di questi strumenti di social networking...,Facebook


In [40]:
# Melting the profilo_value DF --> transposing the data from horizontal to vertical
profilo_value_melt = pd.melt(profilo_value, id_vars='ID_Rispondente', var_name='domanda', value_name='values_risposta')
profilo_value_melt.shape

(3753076, 3)

In [41]:
# Melting the profilo_label DF --> transposing the data from horizontal to vertical
profilo_label_melt = pd.melt(profilo_label, id_vars='ID_Rispondente', var_name='domanda', value_name='labels_risposta')
profilo_label_melt.shape

(3753076, 3)

In [42]:
# Inner joining the profilo_label_melt and profilo_value_melt --> profilo_melt
profilo_melt = pd.merge(profilo_label_melt, profilo_value_melt, how='inner', on=['ID_Rispondente','domanda'])
print(profilo_melt.shape)
profilo_melt.head()

(3753076, 4)


,ID_Rispondente,domanda,labels_risposta,values_risposta
0,256848,B01_Q64_1,Selected,1.0
1,337988,B01_Q64_1,Selected,1.0
2,369486,B01_Q64_1,Selected,1.0
3,371674,B01_Q64_1,Selected,1.0
4,379221,B01_Q64_1,Selected,1.0


In [43]:
# Left joining the profilo_melt with the profilo_lookup (data mapping) DF --> generating the final profilo DF
profilo = pd.merge(profilo_melt, profilo_lookup[['Variabile', 'Domanda', 'Topic']], how='left', left_on='domanda', right_on='Variabile')
print(profilo.shape)
profilo.head()

(3753076, 7)


,ID_Rispondente,domanda,labels_risposta,values_risposta,Variabile,Domanda,Topic
0,256848,B01_Q64_1,Selected,1.0,B01_Q64_1,Quali di questi strumenti di social networking...,Twitter
1,337988,B01_Q64_1,Selected,1.0,B01_Q64_1,Quali di questi strumenti di social networking...,Twitter
2,369486,B01_Q64_1,Selected,1.0,B01_Q64_1,Quali di questi strumenti di social networking...,Twitter
3,371674,B01_Q64_1,Selected,1.0,B01_Q64_1,Quali di questi strumenti di social networking...,Twitter
4,379221,B01_Q64_1,Selected,1.0,B01_Q64_1,Quali di questi strumenti di social networking...,Twitter


In [44]:
# Renaming the variables and dropping unwanted columns
profilo.columns = ['idRispondente', 'cdDomanda', 'cdLabelsRisposta', 'cdValuesRisposta', 'Variabile', 'nmDomanda', 'nmRisposta']
profilo = profilo[['idRispondente', 'cdDomanda', 'cdLabelsRisposta', 'cdValuesRisposta', 'nmDomanda', 'nmRisposta']]
profilo.head()


,idRispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta
0,256848,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
1,337988,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
2,369486,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
3,371674,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
4,379221,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter


In [45]:
# Check size and head of data 
print("Size:", profilo.shape)
profilo.head()

Size: (3753076, 6)


,idRispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta
0,256848,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
1,337988,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
2,369486,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
3,371674,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
4,379221,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter


In [46]:
# filter profilo 'nmDomanda' only for relevat questions for my analysis
profilo = profilo[(profilo['nmDomanda'] =='Hai animali domestici che vivono con te?') |
        (profilo['nmDomanda'] == 'Nel tuo nucleo familiare sono presenti bambini (0-14 anni)? E se sì, quanti')]
profilo = profilo.copy()

In [47]:
print(profilo.shape)
profilo.head()

(25706, 6)


,idRispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta
822592,256848,B03_Q63,"Sì, è presente 1 bambino 0-14 anni",2.0,Nel tuo nucleo familiare sono presenti bambini...,Nel tuo nucleo familiare sono presenti bambini...
822593,337988,B03_Q63,"Sì, è presente 1 bambino 0-14 anni",2.0,Nel tuo nucleo familiare sono presenti bambini...,Nel tuo nucleo familiare sono presenti bambini...
822594,369486,B03_Q63,"No, nel mio nucleo non ci sono bambini 0-14 anni",1.0,Nel tuo nucleo familiare sono presenti bambini...,Nel tuo nucleo familiare sono presenti bambini...
822595,371674,B03_Q63,"Sì, è presente 1 bambino 0-14 anni",2.0,Nel tuo nucleo familiare sono presenti bambini...,Nel tuo nucleo familiare sono presenti bambini...
822596,379221,B03_Q63,"No, nel mio nucleo non ci sono bambini 0-14 anni",1.0,Nel tuo nucleo familiare sono presenti bambini...,Nel tuo nucleo familiare sono presenti bambini...


In [48]:
# check null values
profilo.isnull().sum()

idRispondente          0
cdDomanda              0
cdLabelsRisposta    5170
cdValuesRisposta    5170
nmDomanda              0
nmRisposta             0
dtype: int64

In [49]:
# drop rows with null answers
profilo.dropna(subset='cdLabelsRisposta', axis=0, inplace=True)
profilo.head()

,idRispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta
822592,256848,B03_Q63,"Sì, è presente 1 bambino 0-14 anni",2.0,Nel tuo nucleo familiare sono presenti bambini...,Nel tuo nucleo familiare sono presenti bambini...
822593,337988,B03_Q63,"Sì, è presente 1 bambino 0-14 anni",2.0,Nel tuo nucleo familiare sono presenti bambini...,Nel tuo nucleo familiare sono presenti bambini...
822594,369486,B03_Q63,"No, nel mio nucleo non ci sono bambini 0-14 anni",1.0,Nel tuo nucleo familiare sono presenti bambini...,Nel tuo nucleo familiare sono presenti bambini...
822595,371674,B03_Q63,"Sì, è presente 1 bambino 0-14 anni",2.0,Nel tuo nucleo familiare sono presenti bambini...,Nel tuo nucleo familiare sono presenti bambini...
822596,379221,B03_Q63,"No, nel mio nucleo non ci sono bambini 0-14 anni",1.0,Nel tuo nucleo familiare sono presenti bambini...,Nel tuo nucleo familiare sono presenti bambini...


In [50]:
# pivot profilo table to merge it with the indagine table
profilo_pivot = profilo.pivot(index='idRispondente', columns='nmDomanda', values='cdLabelsRisposta')
profilo_pivot.reset_index(drop=False, inplace=True)

In [51]:
profilo_pivot.head()

nmDomanda,idRispondente,Hai animali domestici che vivono con te?,"Nel tuo nucleo familiare sono presenti bambini (0-14 anni)? E se sì, quanti"
0,256848,Sì,"Sì, è presente 1 bambino 0-14 anni"
1,337988,Sì,"Sì, è presente 1 bambino 0-14 anni"
2,369486,Sì,"No, nel mio nucleo non ci sono bambini 0-14 anni"
3,371674,Sì,"Sì, è presente 1 bambino 0-14 anni"
4,379221,No,"No, nel mio nucleo non ci sono bambini 0-14 anni"


## Merge Indagine DF with Profilo relevant infos to add

In [52]:
# create new final dataframe with indagine details and some profilo details
final_df = pd.merge(indagine, profilo_pivot, how='left', left_on='ID_Rispondente', right_on='idRispondente')

In [53]:
print(final_df.shape)
final_df.head()

(641480, 18)


,ID_Rispondente,Variabile,Domanda,Topic,labels_risposta,values_risposta,Age,CAP,Provincia,Numero di abitanti,Ultimo titolo di studio conseguito,Data e ora di compilazione del questionario,Secondi per risposta,Sesso,peso,idRispondente,Hai animali domestici che vivono con te?,"Nel tuo nucleo familiare sono presenti bambini (0-14 anni)? E se sì, quanti"
0,221250719,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,Molto,5,20,70032,BA,56929,Diploma di scuola media superiore,2023-02-14 19:06:44,10.84925,Un uomo,5.458265,221250719.0,No,"No, nel mio nucleo non ci sono bambini 0-14 anni"
1,89508748,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,Un po' sì e un po' no,3,21,87060,CS,1281,Diploma di scuola media superiore,2023-02-27 18:36:02,12.965,Una donna,5.458265,89508748.0,No,"No, nel mio nucleo non ci sono bambini 0-14 anni"
2,98143773,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,Poco,2,21,63822,FM,16062,Diploma di scuola media superiore,2023-02-19 18:39:59,9.831083,Una donna,5.458265,98143773.0,Sì,"No, nel mio nucleo non ci sono bambini 0-14 anni"
3,641447689,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,Poco,2,21,90024,PA,7614,Diploma di scuola media superiore,2023-02-28 20:32:30,46.2905,Una donna,5.458265,641447689.0,Sì,"Sì, è presente 1 bambino 0-14 anni"
4,61843325,B11_Q62_A_1,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,Poco,2,21,95024,CT,50190,Diploma di scuola media superiore,2023-02-16 18:25:16,10.960417,Una donna,5.458265,61843325.0,No,"No, nel mio nucleo non ci sono bambini 0-14 anni"


In [54]:
final_df.isnull().sum()

ID_Rispondente                                                                     0
Variabile                                                                          0
Domanda                                                                            0
Topic                                                                              0
labels_risposta                                                                    0
values_risposta                                                                    0
Age                                                                                0
CAP                                                                                0
Provincia                                                                      36814
Numero di abitanti                                                                 0
Ultimo titolo di studio conseguito                                                 0
Data e ora di compilazione del questionario                      

## Create a folder and save Final Dataframe

In [55]:
# create directory to save final dataframe
import os
os.makedirs('data_clean', exist_ok=True)
final_df.to_csv('data_clean/01_indagine_final_df.csv', index=False)